In [1]:
#imports
import datetime
import jdcal
import pandas
import io
import json

exo_planet_information = [] #list of dictionaries of exoplanet information

#get getting all of the information from the csv file
def get_all_exo_planet_information():
    data = pandas.read_csv('exo_planets.csv')
    for i in range(len(data["pl_hostname"])):
        planet = {
            'star_host_name': data["pl_hostname"][i],
            'name': data["pl_name"][i],
            'discovery_method': data["pl_discmethod"][i],
            'orbital_period': data["pl_orbper"][i],
            'stellar_distance': data["st_dist"][i],
            'stellar_mass': data["st_mass"][i],
            'stellar_radius': data["st_rad"][i],
            'planet_radius': data["pl_rade"][i],
            'duration': data["pl_trandur"][i],
            'midpoint_julian': data["pl_tranmid"][i],
            'facility_discovered': data["pl_facility"][i],
        }
        exo_planet_information.append(planet)
        
get_all_exo_planet_information()

In [2]:
#converting julian_date to gregorian date
def convert_julian_days_to_legible_date(julian_date):
    
    y, m, d, f = jdcal.jd2jcal(julian_date, 0)
    date_time_obj = datetime.datetime(y,m,d) + datetime.timedelta(f)
    
    return date_time_obj

In [3]:
#methods below compute the next transit given the information of an exoplanet
def compute_next_transit(exoplanet_info):
    transit_start = compute_transit_start_julian(exoplanet_info)
    transit_end = compute_transit_end_julian(exoplanet_info)
    
    delta = compute_transit_delta(exoplanet_info)
    now = datetime.datetime.now()
    
    while(convert_julian_days_to_legible_date(transit_start) < now):
        transit_start += delta
        transit_end += delta
    
    return (convert_julian_days_to_legible_date(transit_start).strftime('%Y %b %d %H:%M:%S.%f'),
            convert_julian_days_to_legible_date(transit_end).strftime('%Y %b %d %H:%M:%S.%f'))
    
def compute_transit_delta(exoplanet_info): 
    return exoplanet_info['orbital_period']

def compute_transit_start_julian(exo_planet_info):
    midpoint = exo_planet_info["midpoint_julian"]
    duration_to_midpoint = exo_planet_info["duration"] / 2
    start_time = midpoint - duration_to_midpoint
    
    return start_time

def compute_transit_end_julian(exo_planet_info):
    midpoint = exo_planet_info["midpoint_julian"]
    duration_to_midpoint = exo_planet_info["duration"] / 2
    start_time = midpoint + duration_to_midpoint
    
    return start_time

temp_info = [(i, compute_next_transit(i)) for i in exo_planet_information]

transit_info = []
for info, transit in temp_info:
    info["next_transit_start"] = transit[0]
    info["next_transit_end"] = transit[1]
    transit_info.append(info)
    
len(transit_info)

1736

In [9]:
f= open("exo_planets.txt","w+")
f.write(str(transit_info))

714816